## Necessary Imports

In [1]:
import json
from dotenv import load_dotenv
from os.path import join, dirname
import os
import requests
import pandas as pd
import numpy as np
from collections import Counter
import watson_developer_cloud
import watson_developer_cloud.natural_language_understanding.features.v1 as features

In [2]:
load_dotenv('.env')

True

## Credentials
Watson Credentials

In [3]:
username=os.environ.get('NLU_USERNAME')

In [4]:
password = os.environ.get('NLU_PASSWORD')

Foursquare Credentials

In [5]:
foursquare_client_id=os.environ.get('FOURSQUARE_CLIENT_ID')

In [6]:
foursquare_client_secret=os.environ.get('FOURSQUARE_CLIENT_SECRET')

Toy variables to mess around with

Lat and long for Galvanize

In [7]:
lat = 40.7263874

In [8]:
long = -74.0076351

In [9]:
VERSION = "20170511"
LIMIT = 30
radius = 1000

In [10]:
with open('venues.json') as jsonfile:
    data = json.load(jsonfile)

In [11]:
data

{'meta': {'code': 200, 'requestId': '5917ab4add579766b5ff9093'},
 'response': {'venues': [{'allowMenuUrlEdit': True,
    'beenHere': {'lastCheckinExpiredAt': 0},
    'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'contact': {'formattedPhone': '(212) 924-9532', 'phone': '2129249532'},
    'hasMenu': True,
    'hasPerk': False,
    'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
    'id': '4bbf6b6430c99c74302a5511',
    'location': {'address': '549 Greenwich St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'distance': 158,
     'formattedAddress': ['549 Greenwich St',
      'New York, NY 10013',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.727468735338

In [12]:
venue_id = "5018507fe4b03a729d0b40f9" # ID of Galli Restaurant

url="https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit=150".format(venue_id, foursquare_client_id, foursquare_client_secret, VERSION)

tips_galli = requests.get(url).json()

In [143]:
def pull_foursquare_json(venue_id):
    VERSION = "20170511"
    foursquare_client_id=os.environ.get('FOURSQUARE_CLIENT_ID')
    foursquare_client_secret=os.environ.get('FOURSQUARE_CLIENT_SECRET')
    url="https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit=150".format(venue_id, foursquare_client_id, foursquare_client_secret, VERSION)
    json_f = requests.get(url).json()
    return json_f

In [13]:
tips_galli

{'meta': {'code': 200, 'requestId': '59185d2adb04f50b150d4084'},
 'response': {'tips': {'count': 123,
   'items': [{'agreeCount': 5,
     'canonicalUrl': 'https://foursquare.com/item/521236c011d25305bc0b8cbc',
     'createdAt': 1376925376,
     'disagreeCount': 0,
     'id': '521236c011d25305bc0b8cbc',
     'lang': 'en',
     'likes': {'count': 6,
      'groups': [{'count': 6,
        'items': [{'firstName': 'Elyse',
          'gender': 'female',
          'id': '90622',
          'lastName': 'Estrada',
          'photo': {'prefix': 'https://igx.4sqi.net/img/user/',
           'suffix': '/VAMV2B2FTCCPXDSU.jpg'}},
         {'firstName': 'Wyatt',
          'gender': 'male',
          'id': '24717007',
          'lastName': 'Savarese',
          'photo': {'prefix': 'https://igx.4sqi.net/img/user/',
           'suffix': '/NSNFLKIII2BOXNGL.jpg'}},
         {'firstName': 'Belle',
          'gender': 'female',
          'id': '2130058',
          'lastName': 'Lenz',
          'photo': {'prefi

In [14]:
len(tips_galli['response']['tips']['items'][:])

123

In [15]:
tips_galli['response']['tips']['count']

123

In [17]:
tips_galli['response']['tips']['items'][1]['text']

"Classic italian done right. The Mad Calamari smothered in spicy marinara is a MUST. Go with the penne w/ vodka sauce or carbonara and you'll leave happy. Pay with cover and skip waiting for the check!"

## Function to get list of tips for a given restaurant

In [18]:
# Helper function to return list of tips from restaurant's JSON file
def tips_list(json_file):
    num_tips = json_file['response']['tips']['count']
    return [json_file['response']['tips']['items'][k]['text'] for k in range(num_tips)]

In [19]:
list_of_tips = tips_list(tips_galli)

In [20]:
venue_ids = ['4bbf6b6430c99c74302a5511', '4baff5d9f964a5207c2e3ce3', '5018507fe4b03a729d0b40f9', '3fd66200f964a5209ae31ee3', '3fd66200f964a520ade61ee3']

## Combine users's preferences
Combine two user's preferences (as defined by tone analysis) into two dicts, combining values if necessary

In [21]:
u_1 = Counter({'sushi':0.7, 'mexican':0.67, 'drinks':0.64, 'korean':0.5, 'russian':0.4, 'whiskey':1, 'tacos':.3})

In [22]:
u_2 = Counter({'whiskey':0.3, 'deli':0.9, 'hamburger':0.99, 'tacos':0.3, 'brewery':0.2})

In [60]:
u_1_dict = {'sushi':0.7, 'mexican':0.67, 'drinks':0.64, 'korean':0.5, 'russian':0.4, 'whiskey':1, 'tacos':.3}

In [92]:
u_2_dict= {'whiskey':0.3, 'deli':0.9, 'hamburger':0.99, 'tacos':0.3, 'brewery':0.2}

In [61]:
u_1_counter = Counter(u_1_dict)

In [62]:
u_1 == u_1_counter

True

In [23]:
u_f = u_1 + u_2

In [24]:
u_f

Counter({'brewery': 0.2,
         'deli': 0.9,
         'drinks': 0.64,
         'hamburger': 0.99,
         'korean': 0.5,
         'mexican': 0.67,
         'russian': 0.4,
         'sushi': 0.7,
         'tacos': 0.6,
         'whiskey': 1.3})

In [48]:
dx = u_f.most_common

In [49]:
factor = 1/sum(u_f.values())

In [50]:
factor

0.14492753623188406

In [58]:
[key for key in u_f.keys()]

['sushi',
 'mexican',
 'drinks',
 'korean',
 'russian',
 'whiskey',
 'tacos',
 'deli',
 'hamburger',
 'brewery']

In [94]:
u_f.most_common(5)

[('whiskey', 1.3),
 ('hamburger', 0.99),
 ('deli', 0.9),
 ('sushi', 0.7),
 ('mexican', 0.67)]

In [59]:
[val for val in u_f.values()]

[0.7, 0.67, 0.64, 0.5, 0.4, 1.3, 0.6, 0.9, 0.99, 0.2]

In [77]:
vallist = [val for val in u_f.values()]

In [79]:
vallist

[0.7, 0.67, 0.64, 0.5, 0.4, 1.3, 0.6, 0.9, 0.99, 0.2]

In [83]:
med = np.median(vallist)

In [84]:
normed_val = [val/med for val in vallist]

In [85]:
normed_val

[1.0687022900763357,
 1.0229007633587786,
 0.97709923664122134,
 0.76335877862595414,
 0.61068702290076338,
 1.9847328244274809,
 0.91603053435114501,
 1.3740458015267176,
 1.5114503816793892,
 0.30534351145038169]

In [131]:
# Function takes in two user preference dicts and outputs indexed pandas vector, index for matrix

from collections import Counter
import pandas as pd

def combine_u_prefs(u1_dict, u2_dict, k):
    u1_counter = Counter(u1_dict)
    u2_counter = Counter(u2_dict)
    u_prefs = u1_counter + u2_counter
    # Pull k highest values
    u_prefs_top = dict(u_prefs.most_common(k))
    vallist = [val for val in u_prefs_top.values()]
    factor = np.median(vallist)
    normed_val = [val/factor for val in vallist]
    topic_idx = [key for key in u_prefs_top.keys()]
    pref_vec = pd.Series(data=normed_val, index=topic_idx)
    return pref_vec, topic_idx

In [132]:
pref_vec, topic_idx = combine_u_prefs(u_1_dict, u_2_dict, 4)

In [134]:
pref_vec

whiskey      1.375661
hamburger    1.047619
deli         0.952381
sushi        0.740741
dtype: float64

In [135]:
topic_idx

['whiskey', 'hamburger', 'deli', 'sushi']

## Get venue list, topic idx return matrix

In [104]:
venue_ids

['4bbf6b6430c99c74302a5511',
 '4baff5d9f964a5207c2e3ce3',
 '5018507fe4b03a729d0b40f9',
 '3fd66200f964a5209ae31ee3',
 '3fd66200f964a520ade61ee3']

In [179]:
def construct_matrix(venue_ids, topic_idx):
    empty_matrix = pd.DataFrame(index=venue_ids, columns=topic_idx)
    return empty_matrix

In [180]:
mat = construct_matrix(venue_ids, topic_idx)

In [181]:
mat

,whiskey,hamburger,deli,sushi
4bbf6b6430c99c74302a5511,NaN,NaN,NaN,NaN
4baff5d9f964a5207c2e3ce3,NaN,NaN,NaN,NaN
5018507fe4b03a729d0b40f9,NaN,NaN,NaN,NaN
3fd66200f964a5209ae31ee3,NaN,NaN,NaN,NaN
3fd66200f964a520ade61ee3,NaN,NaN,NaN,NaN


In [182]:
pref_vec

whiskey      1.375661
hamburger    1.047619
deli         0.952381
sushi        0.740741
dtype: float64

## Get empty matrix, return topic scores

In [183]:
def fill_sentiment_matrix(empty_matrix):

    for j in range(mat.shape[0]):
        venue_id = mat.index[j]
        json_f = pull_foursquare_json(venue_id)
        tips = tips_list(json_f)
        for k in range(mat.shape[1]):
            topic = mat.columns[k]
            score = np.median([sentiment(tip) for tip in tips if topic in tip])
            mat.loc[venue_id, topic] = score
    return mat.fillna(0)

In [184]:
nonzero_mat = fill_sentiment_matrix(empty_matrix=mat)

/home/kt12/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/kt12/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [185]:
nonzero_mat

,whiskey,hamburger,deli,sushi
4bbf6b6430c99c74302a5511,0,0,0.000000,0
4baff5d9f964a5207c2e3ce3,0,0,0.000000,0
5018507fe4b03a729d0b40f9,0,0,0.801052,0
3fd66200f964a5209ae31ee3,0,0,0.944869,0
3fd66200f964a520ade61ee3,0,0,0.932581,0


In [221]:
def recommend(score_mat, user_vec, venues_ids, top_n):
    score_vec = pd.Series(np.dot(score_mat, user_vec), index=venue_ids) 
    return score_vec.sort_values(ascending=False)[:top_n].index.values

In [222]:
s_vec = recommend(nonzero_mat, pref_vec, venue_ids, 3)

In [223]:
s_vec

array(['3fd66200f964a5209ae31ee3', '3fd66200f964a520ade61ee3',
       '5018507fe4b03a729d0b40f9'], dtype=object)

In [224]:
for k in s_vec:
    print(k)

3fd66200f964a5209ae31ee3
3fd66200f964a520ade61ee3
5018507fe4b03a729d0b40f9


In [119]:
mat.index[1]

'4baff5d9f964a5207c2e3ce3'

In [147]:
test_mat = mat

In [150]:
test_mat.loc['4bbf6b6430c99c74302a5511','deli']

nan

In [144]:
test = pull_foursquare_json('4baff5d9f964a5207c2e3ce3')

In [145]:
test

{'meta': {'code': 200, 'requestId': '591871aedd5797667fee2a25'},
 'response': {'tips': {'count': 4,
   'items': [{'agreeCount': 0,
     'canonicalUrl': 'https://foursquare.com/item/4ef907bca69dea74fc14e810',
     'createdAt': 1324943292,
     'disagreeCount': 0,
     'id': '4ef907bca69dea74fc14e810',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': "Tip is included in the bill, regardless of size of group. Odd that they don't mention it upfront. It is 20%.",
     'todo': {'count': 1},
     'type': 'user',
     'user': {'firstName': 'Shai',
      'gender': 'male',
      'id': '65966',
      'lastName': 'Goldman',
      'photo': {'prefix': 'https://igx.4sqi.net/img/user/',
       'suffix': '/65966_1255916715179.jpg'}}},
    {'agreeCount': 0,
     'canonicalUrl': 'https://foursquare.com/item/4e4f07bcd22daf51d270dd18',
     'createdAt': 1313802172,
     'disagreeCount': 0,
     'id': '4e4f07bcd22daf51d270dd18',
     'lang': 'en',
     'likes':

## Get sentiment score from Watson

In [25]:
nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username=username,
                                                            password=password)

In [26]:
output= nlu.analyze(text='this is my experimental text.  Bruce Banner is the Hulk and Bruce Wayne is BATMAN! Superman fears not Banner, but Wayne.',
            features=[features.Sentiment()])

In [27]:
output

{'language': 'en',
 'sentiment': {'document': {'label': 'positive', 'score': 0.922588}}}

In [28]:
output['sentiment']['document']['score']

0.922588

In [29]:
from dotenv import load_dotenv
from os.path import join, dirname
import os
import watson_developer_cloud
import watson_developer_cloud.natural_language_understanding.features.v1 as features

def sentiment(tips):
    # Helper function to return text sentiment analysis
    # Load Watson credentials
    username=os.environ.get('NLU_USERNAME')
    password = os.environ.get('NLU_PASSWORD')
    nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
        username=username, password=password)
    output = nlu.analyze(text=tips, features=[features.Sentiment()])
    return output['sentiment']['document']['score']

In [30]:
sentiment('A great place to come for classic Italian comfort food - the rice balls and meatball sliders are must-order dishes. For those with a gluten allergy, they make delicious gluten-free homemade pastas.')

0.72888

In [67]:
sentiment("I'll only recommend this place if you're around the area and feel the need to try the baos or want that fries fix but I will not be returning due to the fact better baos can be found elsewhere.")

0.0